In [1]:
# mypy: disable-error-code="no-untyped-def, no-untyped-call"
# pyright: reportUnusedExpression=false, reportDuplicateImport=false, reportUnusedImport=false
import os
from pathlib import Path

p = Path.cwd()
for _ in range(100):
    if p.name == "cmomy":
        break
    p = p.parent

os.environ["NUMBA_CACHE_DIR"] = str(p / ".numba_cache")

In [2]:
%%html
<style>
    div.output_stderr {
    display: none;
    }
</style>


In [3]:
import numpy as np

np.set_printoptions(precision=4)

# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [4]:
import numpy as np
import xarray as xr

import cmomy
import cmomy.resample

np.random.seed(0)
cmomy.resample.numba_random_seed(0)

nsamp = 100
energy = xr.DataArray(np.random.rand(nsamp), dims=["samp"])

position = xr.DataArray(np.random.rand(nsamp, 3), dims=["samp", "dim"])

# weight associated with each sample and each record
w = xr.DataArray(np.random.rand(nsamp), dims=["samp"])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim="samp", mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim="samp", mom=3)

In [5]:
print("energy")
ce

energy


<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([4.9972e+01, 4.7186e-01, 7.9673e-02, 2.7133e-03])
Dimensions without coordinates: mom_0

In [6]:
print("position")
cp

position


<xCentralMoments(val_shape=(3,), mom=(3,))>
<xarray.DataArray (dim: 3, mom_0: 4)>
array([[ 4.9972e+01,  5.3881e-01,  8.7283e-02, -7.5029e-03],
       [ 4.9972e+01,  5.0714e-01,  7.7545e-02,  2.6483e-03],
       [ 4.9972e+01,  5.0251e-01,  9.4424e-02, -1.4499e-03]])
Dimensions without coordinates: dim, mom_0

Here, we've used the constructor {meth}`cmomy.CentralMoments.from_vals`.  There are a host of other constructors to create {class}`cmomy.CentralMoments` object.  Take a look at the docs for further info.

## Basic attributes

Notice that there are three `shape` parameters associated with a {class}`cmomy.CentralMoments` object:
 
* {attr}`cmomy.CentralMoments.mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* {attr}`cmomy.CentralMoments.val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vector data, this is the shape of the passed observation data.
* {attr}`cmomy.CentralMoments.shape`: total shape of wrapped moments `shape = val_shape + mom_shape`

In [7]:
for name, c in {"energy": ce, "position": cp}.items():
    print(
        f"""
{name} shapes:
    mom_shape: {c.mom_shape}
    val_shape: {c.val_shape}
    shape    : {c.shape}
"""
    )


energy shapes:
    mom_shape: (4,)
    val_shape: ()
    shape    : (4,)


position shapes:
    mom_shape: (4,)
    val_shape: (3,)
    shape    : (3, 4)



To access the underlying data, use the {attr}`cmomy.CentralMoments.values` attribute.  The structure of `values` is:

* `values[...,0]`: total weight
* `values[...,1]`: average/mean value
* `values[...,k>1]`: `kth` central moment.

In [8]:
ce.values

<xarray.DataArray (mom_0: 4)>
array([4.9972e+01, 4.7186e-01, 7.9673e-02, 2.7133e-03])
Dimensions without coordinates: mom_0

To access all the central moments (zeroth and first included), use the {meth}`cmomy.CentralMoments.cmom` method.

In [9]:
ce.cmom()

<xarray.DataArray (mom_0: 4)>
array([1.    , 0.    , 0.0797, 0.0027])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments using the {meth}`cmomy.CentralMoments.rmom` method.  This uses the {mod}`cmomy.convert` module behind the scenes.  

In [10]:
# <x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)>
array([1.    , 0.4719, 0.3023, 0.2206])
Dimensions without coordinates: mom_0

Additionally, there are {class}`xarray.DataArray` like attributes

In [11]:
ce.coords

Coordinates:
    *empty*

In [12]:
ce.attrs

{}

In [13]:
ce.sizes

Frozen({'mom_0': 4})

In [14]:
ce.dims

('mom_0',)

## Manipulating (co)moments

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [15]:
nsamp = 100
nrec = 10
energy = xr.DataArray(np.random.rand(nrec, nsamp), dims=["rec", "samp"])
position = xr.DataArray(np.random.rand(nrec, nsamp, 3), dims=["rec", "samp", "dim"])

# weight associated with each sample and each record
w = xr.DataArray(np.random.rand(nrec, nsamp), dims=["rec", "samp"])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim="samp", mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim="samp", mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the {meth}`cmomy.xCentralMoments.block` method.

In [16]:
ce

<xCentralMoments(val_shape=(10,), mom=(3,))>
<xarray.DataArray (rec: 10, mom_0: 4)>
array([[ 4.9492e+01,  5.0085e-01,  9.5395e-02,  6.3896e-05],
       [ 4.7085e+01,  4.3387e-01,  8.1477e-02,  8.5081e-03],
       [ 4.5617e+01,  4.5139e-01,  9.1416e-02,  1.0999e-02],
       [ 5.1835e+01,  5.5002e-01,  6.9482e-02, -3.0315e-03],
       [ 4.7215e+01,  5.3124e-01,  8.7490e-02, -3.5280e-03],
       [ 4.7843e+01,  4.7763e-01,  9.5671e-02, -2.0765e-05],
       [ 4.4453e+01,  4.8569e-01,  8.4392e-02,  6.7738e-03],
       [ 5.4015e+01,  5.0624e-01,  9.4580e-02, -9.7587e-04],
       [ 4.8715e+01,  5.4611e-01,  8.5452e-02, -7.4777e-03],
       [ 5.2069e+01,  4.9491e-01,  9.1639e-02,  1.7960e-03]])
Dimensions without coordinates: rec, mom_0

In [17]:
ce.block(block_size=2, dim="rec")

<xCentralMoments(val_shape=(5,), mom=(3,))>
<xarray.DataArray (rec: 5, mom_0: 4)>
array([[ 9.6577e+01,  4.6819e-01,  8.9730e-02,  4.8777e-03],
       [ 9.7451e+01,  5.0385e-01,  8.2171e-02,  1.9050e-03],
       [ 9.5058e+01,  5.0426e-01,  9.2326e-02, -2.0914e-03],
       [ 9.8468e+01,  4.9696e-01,  9.0085e-02,  2.6780e-03],
       [ 1.0078e+02,  5.1966e-01,  8.9303e-02, -2.9227e-03]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data using {meth}`cmomy.xCentralMoments.resample_and_reduce`.  We produce a 20 new samples taken from the original (averaged) data.

In [18]:
ce_resamp = ce.resample_and_reduce(nrep=20, dim="rec")
ce_resamp

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 4.9960e+02,  5.1003e-01,  8.6881e-02, -1.1309e-03],
       [ 4.9127e+02,  5.0051e-01,  8.9029e-02,  4.9401e-04],
       [ 4.9841e+02,  5.1268e-01,  8.8314e-02, -2.0130e-03],
       [ 4.9935e+02,  5.0911e-01,  8.2797e-02, -3.1451e-04],
       [ 5.0143e+02,  4.9972e-01,  9.1087e-02,  1.1648e-03],
       [ 4.7914e+02,  5.0116e-01,  9.0361e-02, -1.7655e-04],
       [ 4.9696e+02,  4.9175e-01,  8.8246e-02,  2.2106e-03],
       [ 4.8638e+02,  5.1882e-01,  8.6051e-02, -1.7109e-03],
       [ 4.8847e+02,  5.0898e-01,  8.9187e-02, -1.7111e-03],
       [ 4.8411e+02,  4.8801e-01,  9.0588e-02,  2.1189e-03],
       [ 4.9949e+02,  4.9524e-01,  9.2165e-02,  1.3359e-03],
       [ 4.8434e+02,  4.9382e-01,  8.5227e-02,  2.8026e-03],
       [ 4.8042e+02,  4.9902e-01,  8.7903e-02,  1.6597e-03],
       [ 4.8683e+02,  5.1949e-01,  8.3902e-02, -1.3177e-03],
       [ 4.9034e+02,  5.2755e-01,  8.5762e-02, -3.4690e-03],
       [ 4.9190e+02,  5.1303e-01,  8.8855e-02, -1.5487e-03],
       [ 4.8523e+02,  5.0076e-01,  8.7806e-02,  7.9204e-04],
       [ 4.8666e+02,  5.0408e-01,  9.0099e-02,  1.1508e-04],
       [ 4.7899e+02,  5.1050e-01,  8.1996e-02,  6.2215e-04],
       [ 4.8574e+02,  5.0812e-01,  8.9516e-02, -1.1245e-03]])
Dimensions without coordinates: rep, mom_0

This is different than the usual 'resample values'.  This is also available if the original data is available.

In [19]:
a = energy.stack(c=["rec", "samp"])

In [20]:
b = w.stack(c=["rec", "samp"])

In [21]:
# consider 'all' the data for this
out = cmomy.xCentralMoments.from_resample_vals(
    energy.stack(c=["rec", "samp"]),
    w=w.stack(c=["rec", "samp"]),
    dim="c",
    nrep=20,
    mom=3,
)
out

<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 4.9870e+02,  4.9783e-01,  9.2042e-02, -9.4313e-05],
       [ 4.8287e+02,  4.8787e-01,  8.7446e-02,  2.8642e-03],
       [ 4.9105e+02,  4.9118e-01,  8.6676e-02,  1.3127e-03],
       [ 4.8382e+02,  4.9930e-01,  8.5147e-02,  1.3130e-03],
       [ 4.8333e+02,  5.0413e-01,  8.6954e-02, -1.0009e-03],
       [ 4.7435e+02,  4.8404e-01,  8.7935e-02,  2.6536e-03],
       [ 4.8042e+02,  5.1110e-01,  8.5659e-02, -8.7397e-04],
       [ 4.8697e+02,  4.7904e-01,  8.5521e-02,  3.5398e-03],
       [ 4.8203e+02,  4.9992e-01,  9.1893e-02,  4.5804e-04],
       [ 4.8522e+02,  4.8408e-01,  8.8583e-02,  2.1770e-03],
       [ 4.8171e+02,  5.0057e-01,  8.6363e-02,  1.4176e-04],
       [ 4.9409e+02,  4.9405e-01,  8.9243e-02,  9.4614e-04],
       [ 4.8227e+02,  4.8302e-01,  8.2409e-02,  1.8896e-03],
       [ 4.9892e+02,  4.8142e-01,  8.9042e-02,  3.7596e-03],
       [ 4.9887e+02,  5.1086e-01,  8.6601e-02,  1.0227e-03],
       [ 4.8603e+02,  4.9589e-01,  8.4435e-02,  2.0395e-03],
       [ 4.9512e+02,  4.9101e-01,  9.0130e-02,  2.3995e-03],
       [ 4.9532e+02,  5.0001e-01,  9.2318e-02,  2.1699e-03],
       [ 4.8511e+02,  4.9988e-01,  8.4595e-02,  1.0280e-03],
       [ 4.7802e+02,  4.9935e-01,  9.7890e-02,  5.3348e-04]])
Coordinates:
    c        object (0, 0)
    rec      int64 0
    samp     int64 0
Dimensions without coordinates: rep, mom_0

We see that the deviation in the moments is similar using the two resampling methods:

In [22]:
out.values.sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)>
array([0.0091, 0.0035, 0.0013])
Coordinates:
    c        object (0, 0)
    rec      int64 0
    samp     int64 0
Dimensions without coordinates: mom_0

In [23]:
ce_resamp.values.sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)>
array([0.0098, 0.0027, 0.0016])
Dimensions without coordinates: mom_0

We can also reduce our original data across all the records using {meth}`cmomy.xCentralMoments.reduce`

In [24]:
ce.reduce(dim="rec")

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([4.8834e+02, 4.9875e-01, 8.8997e-02, 8.5016e-04])
Dimensions without coordinates: mom_0